Import some necessary libraries

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import datetime
import math
import random

Debugging and customizing receipt template size

In [ ]:
def print_centered(text, width=44):
    truncated_text = text[:width]
    print(truncated_text.center(width))

In [ ]:
def print_product_item(name, amount, price):
  print(f'{name[:20].ljust(20)}{str(amount).rjust(5)}{str(price).rjust(7)}{str(price * amount).rjust(10)}')

In [ ]:
print_centered('KS TUBUN 0293493733')
print_centered('JL KS TUBUN NO.05 RT02 RW04 KEL TEMANGGUNG')
print_centered('KEC TEMANGGUNG KAB TEMANGGUNG, 56213')

print('  ', end='')
print('-' * 40)
print('  ')

print(' ', end='')
print_product_item(
    name='K/API KOPI SPC.60GR',
    amount=2,
    price=5500
)

            KS TUBUN 0293493733             
 JL KS TUBUN NO.05 RT02 RW04 KEL TEMANGGUNG 
    KEC TEMANGGUNG KAB TEMANGGUNG, 56213    
  ----------------------------------------
  
 K/API KOPI SPC.60GR     2   5500     11000


Function to generate receipts using the pillow library with precision so that it is similar to the original receipt

In [ ]:
from ctypes import resize
def generate_receipt(product_items, output_filepath):
  font_size = 20
  line_height = 28

  # menggunakan custom font monspace
  try:
      font = ImageFont.truetype("/content/RobotoMono-VariableFont_wght.ttf", size=font_size)
  except IOError:
      font = ImageFont.load_default()

  # mendapatkan ukuran 1 karakter
  char_l, char_t, char_r, char_b = font.getbbox('A')
  char_width = char_r - char_l


  def get_formatted_number(value):
    return "{:,}".format(value)

  def print_header(draw, x, y):
    def print_header_line(y):
      draw.text(
          (2 * char_width, y),
          '-' * 40,
          font=font,
          fill='black',
      )

    def print_center_text(y, text):
      draw.text(
          (x, y),
          text[:42].center(44),
          font=font,
          fill='black',
      )

    texts = [
        'KS TUBUN 0293493733',
        'JL KS TUBUN NO.05 RT02 RW04 KEL TEMANGGUNG',
        'KEC TEMANGGUNG KAB TEMANGGUNG, 56213',
    ]

    for idx, text in enumerate(texts):
      print_center_text(y + idx * line_height, text)

    kasir_names = [
        'RIZAL',
        'DWI',
        'ANGGA',
        'ENO',
        'RETNO',
        'PINKA',
        'TIWI',
        'KARREN',
        'NISA',
        'ELLA',
        'IBAN',
        'NUR',
        'HUDA',
        'IBRA',
        'GITUL',
        'GITA',
        'DONA',
        'SATRIO'
    ]
    print_header_line(y=y + 3 * line_height)
    print_center_text(
        y=y + 4 * line_height,
        text=f'{datetime.datetime.now().strftime("%d.%m.%y-%H:%M")}'
        f'/3.0.20/FXBA-{random.randint(1000, 9999):04d}/'
        f'{random.choice(kasir_names)}/'
        f'{random.randint(1, 5):02d}',
        )
    print_header_line(y=y + 5 * line_height)


  def print_product_item(draw, x, y, name, amount, price):
    draw.text(
        (x, y),
        f'{name[:20].ljust(20)}'
        f'{str(amount).rjust(5)}'
        f'{str(price).rjust(7)}'
        f'{get_formatted_number(price * amount).rjust(10)}',
        font=font, fill='black',
    )

  def print_harga_jual(draw, x, y, total):
    draw.text(
        (char_width, y),
        ('-' * 21).rjust(42),
        font=font, fill='black',
    )
    draw.text(
        (x, y + line_height),
        f'HARGA JUAL :{get_formatted_number(total).rjust(10)}'.rjust(42),
        font=font, fill='black',
    )
    draw.text(
        (char_width, y + line_height * 2),
        ('-' * 21).rjust(42),
        font=font, fill='black',
    )

  def print_summary(draw, x, y, total, tunai):
    titles = ['TOTAL', 'TUNAI', 'KEMBALI']
    values = [total, tunai, tunai - total]

    for idx, title in enumerate(titles):
      draw.text(
          (x, y + idx * line_height),
          f'{title} :{get_formatted_number(values[idx]).rjust(10)}'.rjust(42),
          font=font, fill='black',
      )

  def print_footer(draw, x, y):
    texts = [
        'LAYANAN KONSUMEN',
        'SMS/WA 0811.1500.280 TELP 1500280',
        'KONTAK@INDOMARET.CO.ID',
        'BELANJA LEBIH MUDAH DI KLIKINDOMARET',
        'GRATIS ONGKIR 1 JAM SAMPAI',
    ]

    for idx, text in enumerate(texts):
      draw.text(
          (x, y + idx * line_height),
          text.center(44),
          font=font,
          fill='black',
      )


  # menghitung total tinggi gambar
  header_line_count = 3
  date_line_count = 3
  product_item_line_count = len(product_items)
  harga_jual_line_count = 3
  summary_line_count = 3
  ppn_line_count = 2
  footer_line_count = 5

  image = Image.new("RGB", (char_width * 44, (
      (line_height * 5) +
      ((
          header_line_count +
          date_line_count +
          product_item_line_count +
          harga_jual_line_count +
          summary_line_count +
          ppn_line_count +
          footer_line_count
        ) * line_height) +
      (line_height * 2)
  )), "white")
  draw = ImageDraw.Draw(image)

  # logo
  logo_image = Image.open('/content/Logo_Indomaret.png').convert('L')
  new_logo_width = 18 * char_width
  new_logo_aspect_ratio = logo_image.height / logo_image.width
  new_logo_height = int(new_logo_width * new_logo_aspect_ratio)
  logo_image = logo_image.resize((new_logo_width, new_logo_height))
  image.paste(logo_image, ((44 - 19) * char_width, line_height))

  # header
  y_offset = line_height * 5
  print_header(draw, x=0, y=y_offset)

  # product items
  y_offset += (header_line_count + date_line_count) * line_height
  for idx, product_item in enumerate(product_items):
    print_product_item(
        draw,
        x=char_width,
        y=y_offset + idx * line_height,
        name=product_item['name'],
        amount=product_item['amount'],
        price=product_item['price'],
    )

  # harga jual
  harga_jual = sum([
          (product_item['price'] * product_item['amount'])
          for product_item in product_items])
  y_offset += len(product_items) * line_height
  print_harga_jual(
      draw,
      x=char_width,
      y=y_offset,
      total=harga_jual,
  )

  # ringkasan
  multiple = 5000
  tunai = math.ceil(harga_jual / multiple) * multiple

  y_offset += harga_jual_line_count * line_height
  print_summary(draw,
                x=char_width,
                y=y_offset,
                total=harga_jual,
                tunai=tunai,
                )

  # footer
  y_offset += (summary_line_count + ppn_line_count) * line_height
  print_footer(draw, x=0, y=y_offset)

  # simpan gambar dalam bentuk file
  image.save(output_filepath)
  print("Gambar berhasil disimpan sebagai 'output.png'")

# generate_receipt([
#     {'name': 'PRODUCT NAME 1', 'amount': 2, 'price': 6000},
#     {'name': 'PRODUCT NAME 2', 'amount': 3, 'price': 6500},
#     {'name': 'PRODUCT NAME 3', 'amount': 1, 'price': 8700},
#     {'name': 'PRODUCT NAME 4', 'amount': 2, 'price': 5200},
#     {'name': 'PRODUCT NAME 5', 'amount': 5, 'price': 3200},
# ], 'output.png')

The receipt generation process uses the dataset that has been obtained in the previous notebook. Product items are taken randomly and will be generated as many as 500 receipts.

In [ ]:
import pandas as pd
import re
import os

if not os.path.exists('receipts'):
  os.makedirs('receipts')

df = pd.read_csv('/content/data_receipt_generator.csv')

def get_random_products():
  random_rows = df.sample(n=random.randint(1, 10))
  filtered_rows = random_rows[random_rows.iloc[:, 2].str.contains('Rp', na=False)]
  return filtered_rows

for index in range(0, 500):
  is_found = False
  while not is_found:
    products = get_random_products()
    if len(products) > 0:
      is_found = True

    mapped_products = []
    for idx, row in products.iterrows():
      product_name = row.iloc[0]
      product_name = product_name.upper()

      price = row.iloc[2]
      price = re.sub(r'\D', '', price)
      price = int(price)

      if price < 50000:
        amount = random.randint(1, 10)
      else:
        amount = random.randint(1, 3)

      mapped_products.append({
          'name': product_name,
          'amount': amount,
          'price': price,
      })

    generate_receipt(mapped_products, os.path.join('receipts', f'receipt-{(index):03d}.png'))

  print('-' * 10)

Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.png'
----------
Gambar berhasil disimpan sebagai 'output.p

In [ ]:
!zip -r "receipts.zip" "receipts"

  adding: receipts/ (stored 0%)
  adding: receipts/receipt-181.png (deflated 13%)
  adding: receipts/receipt-358.png (deflated 12%)
  adding: receipts/receipt-275.png (deflated 14%)
  adding: receipts/receipt-221.png (deflated 13%)
  adding: receipts/receipt-266.png (deflated 13%)
  adding: receipts/receipt-290.png (deflated 13%)
  adding: receipts/receipt-011.png (deflated 13%)
  adding: receipts/receipt-392.png (deflated 13%)
  adding: receipts/receipt-477.png (deflated 14%)
  adding: receipts/receipt-485.png (deflated 13%)
  adding: receipts/receipt-239.png (deflated 13%)
  adding: receipts/receipt-470.png (deflated 13%)
  adding: receipts/receipt-253.png (deflated 13%)
  adding: receipts/receipt-320.png (deflated 13%)
  adding: receipts/receipt-424.png (deflated 13%)
  adding: receipts/receipt-457.png (deflated 14%)
  adding: receipts/receipt-449.png (deflated 13%)
  adding: receipts/receipt-385.png (deflated 13%)
  adding: receipts/receipt-463.png (deflated 14%)
  adding: receipts